In [2]:
import pandas as pd
import numpy as np
import os

import sklearn

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

## 검증
테스트 셋으로 일반화 성능을 올바르게 예측하려면 가능한 한 테스트 셋을 사용하지 말아야 한다.  
모델을 만들고나서 마지막에 딱 한 번만 사용하는게 좋다!
  
    
테스트 셋을 사용하지 않고 모델이 과대/과소적합인지 판단하는 방법으로는
Validation Set 을 하나 더 만드는 게 있따!!!

In [3]:
df = pd.read_csv('https://bit.ly/wine-date')
df.shape

(6497, 4)

In [4]:
data = df[['alcohol', 'sugar', 'pH']].to_numpy()
target = df['class'].to_numpy()

In [5]:
train_input, test_input, train_target, test_target = \
    train_test_split(data, target, test_size=0.2, random_state = 42)

In [6]:
sub_input, val_input, sub_target, val_target = \
    train_test_split(train_input, train_target, test_size=0.2, random_state = 42)

In [7]:
sub_input.shape, val_input.shape, test_input.shape

((4157, 3), (1040, 3), (1300, 3))

In [8]:
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)

DecisionTreeClassifier(random_state=42)

In [9]:
# 트레인셋에 과대적합 되어 있으므로 매개변수를 바꾸어 더 좋은 모델을 찾아야 한다!!
dt.score(sub_input, sub_target), dt.score(val_input, val_target)

(0.9971133028626413, 0.864423076923077)

## 교차검증 Cross Validation
- 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다  
- 트레인셋을 섞으려면 분할기 splitter 를 지정해야 한다!    


`cross_validate()` 함수의 전신인 `cross_val_score()` 함수도 있지만, 이 함수는 `cross_validate()` 함수 결과 중 `test_score` 값만을 반환한다.

In [10]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)

In [12]:
# fit_time & score_time > 각 모델을 훈련하는 시간과 검증하는 시간
# len(arr) > n fold
# test_score : 최종 점수는 해당 키의 밸류 값의 평균으로 얻을 수 있다! 
# 이름은 test_score 지만, 검증 폴드의 점수이다!!!
scores

{'fit_time': array([0.0042119 , 0.00416136, 0.00434232, 0.00427246, 0.00411081]),
 'score_time': array([0.00039554, 0.00035191, 0.00035048, 0.00035167, 0.00034356]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [13]:
np.mean(scores['test_score'])

0.855300214703487

In [14]:
from sklearn.model_selection import StratifiedKFold

In [15]:
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
np.mean(scores['test_score'])

0.855300214703487

In [18]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
scores['test_score'], np.mean(scores['test_score'])
# n_splits=10 이라서 scores의 value arr 가 다 10개

(array([0.83461538, 0.87884615, 0.85384615, 0.85384615, 0.84615385,
        0.87307692, 0.85961538, 0.85549133, 0.85163776, 0.86705202]),
 0.8574181117533719)

### Grid Search
( max_depth 와 min_samples_split 등의 ) 매개변수를 동시에 바꾸어가며 최적의 값을 찾아야할 때 사용하는 클래스!

In [19]:
from sklearn.model_selection import GridSearchCV
params = { "min_impurity_decrease" : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [20]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs = -1)

In [21]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [22]:
dt = gs.best_estimator_

In [23]:
dt.score(train_input, train_target)

0.9615162593804117

In [24]:
gs.best_params_, gs.best_score_

({'min_impurity_decrease': 0.0001}, 0.8681929740134745)

In [25]:
gs.cv_results_

{'mean_fit_time': array([0.00489054, 0.00427427, 0.00421157, 0.00391469, 0.00369563]),
 'std_fit_time': array([4.45512726e-04, 5.95830928e-04, 4.50313561e-05, 1.74366029e-04,
        2.80656670e-04]),
 'mean_score_time': array([0.00060015, 0.00054259, 0.00057883, 0.00055842, 0.00050383]),
 'std_score_time': array([9.48683555e-05, 8.53740146e-05, 2.66250259e-05, 2.39903594e-05,
        7.17333953e-05]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004},
  {'min_impurity_decrease': 0.0005}],
 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]),
 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]),
 'split2_test_sc

In [27]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수가 저장되어 있다!
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [29]:
gs.cv_results_['params'][np.argmax(gs.cv_results_['mean_test_score'])]

{'min_impurity_decrease': 0.0001}

In [30]:

params = { "min_impurity_decrease" : np.arange(0.0001, 0.001, 0.0001),
         'max_depth' : range(5, 20, 1),
         'min_samples_split' : range(2, 100, 10) }

In [31]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [32]:
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [33]:
np.max(gs.cv_results_['mean_test_score'])

0.8683865773302731

#### 랜덤 서치
1. 매개 변수의 값이 수치일 때 값의 범위나 간격을 미리 정할 수 없을 때
2. 너무 많은 매개 변수 조건이 있어 그리드서치 수행시간이 오래 걸릴 때

In [36]:
# 균등 분포에서 샘플링하기 위해서 uniform 클래스 사용
from scipy.stats import uniform, randint

In [45]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.58233715, 0.14385417, 0.16504189, 0.37111428, 0.80710367,
       0.92041287, 0.95701114, 0.85957392, 0.90458095, 0.51075253])

In [47]:
# min_samples_leaf : 리프 노드가 되기 위한 최소 샘플 수
params = { "min_impurity_decrease" : uniform(0.0001, 0.001),
         'max_depth' : randint(20, 50),
         'min_samples_split' : randint(2, 25),
         'min_samples_leaf' : randint(1, 25)}

In [48]:
from sklearn.model_selection import RandomizedSearchCV
# n_iter 랜덤 서치 클래스의 샘플링 횟수

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter=100, n_jobs = -1, random_state=42)

In [49]:
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb20cb6d340>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb20c4c2280>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb20c97e400>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb20c97caf0>},
                   random_state=42)

In [50]:
gs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [51]:
np.max(gs.cv_results_['mean_test_score'])

0.8695428296438884

In [52]:
# 테스트셋 점수는 검증 셋 점수보다 조금 작은 게 일반적이다
dt = gs.best_estimator_
dt.score(test_input, test_target)

0.86

In [53]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42, splitter='random'), params, n_iter=100, n_jobs = -1, random_state=42, )

In [54]:
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb20cb6d340>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb20c4c2280>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb20c97e400>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb20c97caf0>},
                   random_state=42)

In [55]:
gs.best_params_

{'max_depth': 43,
 'min_impurity_decrease': 0.00011407982271508446,
 'min_samples_leaf': 19,
 'min_samples_split': 18}

In [56]:
np.max(gs.cv_results_['mean_test_score'])

0.8458726956392981

In [57]:
dt = gs.best_estimator_
dt.score(test_input, test_target)

0.786923076923077